In [ ]:
!pip install --upgrade pip



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install mediapipe-model-maker

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.5/117.5 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.9/33.9 MB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 78.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.1/175.1 kB 9.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 591.0/591.0 kB 55.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.6/240.6 kB 31.1 MB/s eta 0:00:00
 

In [ ]:
from google.colab import files
import os
import tensorflow as tf
assert tf.__version__.startswith('2')
import cv2

from mediapipe_model_maker import gesture_recognizer

import matplotlib.pyplot as plt


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
dataset_path = '/content/gdrive/My Drive/image_subset'


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
print(dataset_path)
# Specify the rotation angle
angle = 90

# Rotate the dataset images
for label in os.listdir(dataset_path):
    label_path = os.path.join(dataset_path, label)
    if os.path.isdir(label_path):
        for filename in os.listdir(label_path):
            if filename.endswith('.jpg') or filename.endswith('.png'):
                image_path = os.path.join(label_path, filename)
                image = cv2.imread(image_path)
                rotated_image = cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE)  # Rotate the image

                rotated_image_path = os.path.join(label_path, 'rotated', filename)
                cv2.imwrite(rotated_image_path, rotated_image)

# Get the labels from the rotated dataset
labels = [label for label in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, label))]
print(labels)


/content/gdrive/My Drive/image_subset
['none', 'Bedroom2Light_OFF', 'LivingRoomLight_OFF', 'AC_ON', 'Bedroom1Light_OFF', 'Bedroom2Light_ON', 'AC_OFF', 'Dishwasher_ON', 'Dishwasher_OFF', 'LivingRoomFan_OFF', 'Bedroom1Light_ON', 'LivingRoomFan_ON', 'LivingRoomBlinds_ON', 'LivingRoomBlinds_OFF', 'LivingRoomLight_ON']


In [ ]:
#This takes the gestures from dataset and mediapipe deciphers the gestures from the provided pictures
data = gesture_recognizer.Dataset.from_folder(
    dirname=dataset_path,
    hparams=gesture_recognizer.HandDataPreprocessingParams()
)

#split the data into training and test data
train_data, rest_data = data.split(0.8)
validation_data, test_data = rest_data.split(0.5)


Using existing files at /tmp/model_maker/gesture_recognizer/palm_detection_full.tflite
Using existing files at /tmp/model_maker/gesture_recognizer/hand_landmark_full.tflite
Using existing files at /tmp/model_maker/gesture_recognizer/gesture_embedder


In [ ]:
#hparams = gesture_recognizer.HParams(export_dir='/content/gdrive/My Drive/exported_model')
hparams = gesture_recognizer.HParams(learning_rate= 0.001, batch_size=256, epochs=200, shuffle = True, export_dir='/content/gdrive/My Drive/exported_model')
options = gesture_recognizer.GestureRecognizerOptions(hparams=hparams)
model = gesture_recognizer.GestureRecognizer.create(
    train_data=train_data,
    validation_data=validation_data,
    options=options
)


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hand_embedding (InputLayer)  [(None, 128)]            0         
                                                                 
 batch_normalization (BatchN  (None, 128)              512       
 ormalization)                                                   
                                                                 
 re_lu (ReLU)                (None, 128)               0         
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 custom_gesture_recognizer_o  (None, 15)               1935      
 ut (Dense)                                                      
                                                                 
Total params: 2,447
Trainable params: 2,191
Non-trainable par

# New Section

In [ ]:
loss, acc = model.evaluate(test_data, batch_size=1)
print(f"Test loss:{loss}, Test accuracy:{acc}")


123/123 [==============================] - 1s 2ms/step - loss: 0.4515 - categorical_accuracy: 0.8130
Test loss:0.4514571726322174, Test accuracy:0.8130081295967102


In [ ]:
model.export_model()
!ls exported_model

ls: cannot access 'exported_model': No such file or directory
